In [ ]:
!pip install google-genai pydantic

In [4]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [5]:
import os
from google import genai
from pydantic import BaseModel

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()

# Set the model we want our agents to use
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define the Structured Schema for the Critic ---
class EvaluationResult(BaseModel):
    is_approved: bool
    feedback: str

# --- 2. Define Subagents ---

def task_a_writer_agent(topic: str, previous_feedback: str = "") -> str:
    """Subagent A: Drafts or refines the content."""
    print("-> [Writer Agent] Drafting content...")

    if previous_feedback:
         prompt = f"Rewrite and improve the content about '{topic}' by applying this feedback: {previous_feedback}"
    else:
         prompt = f"Write a brief, one-sentence promotional hook for: {topic}"

    response = client.models.generate_content(model=MODEL_ID, contents=prompt)
    return response.text

def task_b_critic_agent(content: str) -> EvaluationResult:
    """Subagent B: Reviews the content and returns a strict Pass/Fail state."""
    print("-> [Critic Agent] Evaluating content...")

    prompt = f"""Evaluate the following content. It MUST be exactly one sentence long and sound exciting.
    If it meets the criteria, set is_approved to true and leave feedback empty.
    If it fails, set is_approved to false and provide specific feedback on how to fix it.

    Content: {content}"""

    # We use structured outputs so the orchestrator can easily parse the exact state
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': EvaluationResult,
            'temperature': 0.1, # Low temperature for more consistent grading
        },
    )
    # The SDK automatically parses the JSON into our Pydantic EvaluationResult object
    return response.parsed

# --- 3. Define the Orchestrator ---

def loop_workflow_agent(user_topic: str, max_iterations: int = 3) -> str:
    """
    The orchestrator agent.
    Runs the sequence in a loop until the critic approves or max_iterations is hit.
    """
    print(f"Starting Loop Workflow for: '{user_topic}'\n")

    iteration = 1
    feedback = ""
    current_draft = ""

    # Safeguard: The loop runs until an exit condition is satisfied, but we
    # use max_iterations to prevent infinite loops and excessive resource consumption.
    while iteration <= max_iterations:
        print(f"--- Iteration {iteration} ---")

        # 1. Execute Writer Agent (Task A)
        current_draft = task_a_writer_agent(user_topic, feedback)
        print(f"Draft: {current_draft}")

        # 2. Execute Critic Agent (Task B)
        evaluation = task_b_critic_agent(current_draft)

        # 3. Evaluate Exit Condition
        if evaluation.is_approved:
            print("-> [Loop Agent] ✅ Exit condition satisfied! Content approved.\n")
            return current_draft
        else:
            print(f"-> [Loop Agent] ❌ Exit condition failed. Feedback: {evaluation.feedback}\n")
            # Pass the feedback back to the top of the loop
            feedback = evaluation.feedback
            iteration += 1

    # Alternative Exit Condition: Max iterations reached
    print("-> [Loop Agent] ⚠️ Max iterations reached. Exiting to prevent infinite loop.\n")
    return current_draft

# --- Execution ---
if __name__ == "__main__":
    # The user's request
    topic = "A new AI-powered robotic vacuum cleaner"

    # Trigger the loop agent
    final_output = loop_workflow_agent(topic)

    print("=== Final Approved Output ===")
    print(final_output)

Starting Loop Workflow for: 'A new AI-powered robotic vacuum cleaner'

--- Iteration 1 ---
-> [Writer Agent] Drafting content...
Draft: Here are a few options, choose the one that best fits your brand's tone:

**Option 1 (Focus on Effortless Cleanliness):**
"Experience a consistently spotless home, powered by an AI robotic vacuum that intelligently cleans so you don't have to lift a finger."

**Option 2 (Focus on Intelligence):**
"The future of clean is here: an AI robotic vacuum that intelligently learns, adapts, and meticulously maintains your home with zero effort from you."

**Option 3 (Concise & Direct):**
"Meet the new AI robotic vacuum that intelligently cleans your home to perfection, making spotless floors effortless."
-> [Critic Agent] Evaluating content...
-> [Loop Agent] ❌ Exit condition failed. Feedback: The provided content is not a single sentence; it includes an introductory phrase and three distinct options. Please provide content that is exactly one sentence long.

--